In [ ]:
#测试输出
train1 = train1.values
train1_values = train1.astype('float32')     # 将整型变为float
scaler_train = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
train1_scaler = scaler_train.fit_transform(train1_values)

# inds = np.arange(train1_scaler.shape[0])
# # random.shuffle(inds)
# # train1_inds = inds[0:int(train1_scaler.shape[0]*0.8)]
# # eval1_inds = inds[int(train1_scaler.shape[0]*0.8):train1_scaler.shape[0]]


train_x, train_y = train1_scaler[:,:-1], train1_scaler[:,-1:]

#测试输出
train1=pd.DataFrame(train1)
train1_values=pd.DataFrame(train1_values)
train1_scaler=pd.DataFrame(train1_scaler)
train_x=pd.DataFrame(train_x)
train_y=pd.DataFrame(train_y)

train1.to_csv('../temp/train1_values.csv',index=False)
train1_values.to_csv('../temp/train1_values.csv',index=False)
train1_scaler.to_csv('../temp/train1_scaler.csv',index=False)

train_x.to_csv('../temp/train_x.csv',index=False)
train_y.to_csv('../temp/train_y.csv',index=False)

In [ ]:
# # convert series to supervised learning
# def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
#     n_vars = 1 if type(data) is list else data.shape[1]
#     df = DataFrame(data)
#     cols, names = list(), list()
#     # input sequence (t-n, ... t-1)
#     for i in range(n_in, 0, -1):
#         cols.append(df.shift(i))
#         names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
#     # forecast sequence (t, t+1, ... t+n)
#     for i in range(0, n_out):
#         cols.append(df.shift(-i))
#         if i == 0:
#             names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
#         else:
#             names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
#     # put it all together
#     agg = concat(cols, axis=1)
#     agg.columns = names
#     # drop rows with NaN values
#     if dropnan:
#         agg.dropna(inplace=True)
#     return agg

In [ ]:
# import datetime
# now = datetime.datetime.now()
# print (now.strftime("%Y-%m-%d %H:%M:%S"))
# predict_1AM1 = model.predict(x=test_x,batch_size=100)
#print(len(predicition))

# XXXXXXXXXXXXXXXXXXXX分割线XXXXXXXXXXXXXXXXXXXXXXXXXXXX

# 反归一化和loss图 及预测图笔记

In [ ]:
# 预测训练集和测试集
trainPredict = model.predict(train_x)
testPredict = model.predict(test_x)

# 反归一化训练集
inv_trainPredict = concatenate((train_x[:,:],trainPredict), axis=1)
inv_trainPredict = scaler.inverse_transform(inv_trainPredict)
trainPredict = inv_trainPredict[:,-1:]
# invert scaling for actual
train_y = train_y.reshape((len(train_y), 1))
inv_trainY = concatenate((train_x[:,:],train_y), axis=1)
inv_trainY = scaler.inverse_transform(inv_trainY)
trainY = inv_trainY[:,-1:]
#反归一化测试集
inv_testPredict = concatenate((test_x[:,:],testPredict), axis=1)
inv_testPredict = scaler.inverse_transform(inv_testPredict)
testPredict = inv_testPredict[:,-1:]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_testY = concatenate((test_x[:,:],test_y), axis=1)
inv_testY = scaler.inverse_transform(inv_testY)
testY = inv_testY[:,-1:]
# 计算训练集和测试集的平均根方误差
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
#训练集和测试集 预测结果保存
predict_1AM2 = concatenate((inv_testY[:,:],testPredict), axis=1)
predict_1AM2 = pd.DataFrame(predict_1AM2)
predict_1AM2.columns = ['Year', 'Month', 'Day', 'Hour', 'Minutes', 'WTC', 'WRH', 'GHR', 'DHR',
       'WD', 'WDR', 'RGT', 'RDT', '1A Trina_power','Prediction 1A Trina_power']
predict_1AM2.to_csv('../predict/predict_1AM2.csv',index=False)
#真实值和预测值对比图
plt.plot(testY,label='Real')
plt.plot(testPredict,label='M2predicition')
# plt.plot(y_rr_,label='rr')
# plt.plot(y_lasso_,label='lasso')
plt.legend() 

In [ ]:
mse_test=np.sum((test_y - testPredict)**2)/ len(test_y)
rmse_test=np.sqrt(mse_test)
print("MSE:",mse_test)
print("RMSE:",rmse_test)

# XXXXXXXXXXX分割线XXXXXXXXXXXXX

In [ ]:
##画曲线
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import spline
 
T = np.array([6, 7, 8, 9, 10, 11, 12])
power = np.array([1.53E+03, 5.92E+02, 2.04E+02, 7.24E+01, 2.72E+01, 1.10E+01, 4.70E+00])
xnew = np.linspace(T.min(),T.max(),300) #300 represents number of points to make between T.min and T.max
 
power_smooth = spline(T,power,xnew)
 
plt.plot(xnew,power_smooth)
plt.show()